In [2]:
!pip show numpy

Name: numpy
Version: 1.22.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /home/lucas/.local/lib/python3.9/site-packages
Requires: 
Required-by: umap-learn, transformers, torchvision, sentence-transformers, seaborn, scipy, scikit-learn, pandas, numba, matplotlib, hdbscan, gym, bertopic


In [3]:
import pkg_resources
pkg_resources.require("NumPy==1.22.4")
import numpy

In [4]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
from bertopic import BERTopic



def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df

tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
#for index, row in tweets.iterrows():
#
#    candidato = ''
#    match row.author_id:
#        case 2670726740:
#            candidato = 'lula'
#        case 128372940:
#            candidato = 'bolsonaro'
#        case 33374761:
#            candidato = 'ciro'
#        case _:
#            candidato: 'n/d'
#    tweets.at[index, 'candidato'] = candidato

tweets_lula = tweets[tweets['author_id'] == 2670726740]
tweets_lula['candidato'] = 'lula'

tweets_bolsonaro = tweets[tweets['author_id'] == 128372940]
tweets_bolsonaro['candidato'] = 'bolsonaro'

tweets_ciro = tweets[tweets['author_id'] == 33374761]
tweets_ciro['candidato'] = 'ciro'

tweets = pd.concat([tweets_lula, tweets_bolsonaro, tweets_ciro], axis="index")

# Embaralhar tweets
tweets = tweets.sample(frac=1)
        

print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()
 
tweets_topic, probabilities = model.fit_transform(docs)


topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index + 1, 'topico'] = topic_names[tweet_topic + 1]

tweets.head()



freq = model.get_topic_info()
print(freq)





/home/lucas/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Número de tweets: 300

      author_id      conversation_id                created_at  \
252   128372940  1557892369990795264 2022-08-12 00:51:00+00:00   
167    33374761  1559374204163743744 2022-08-16 02:59:16+00:00   
6    2670726740  1560226220368969728 2022-08-18 11:25:35+00:00   
176    33374761  1559360566141001728 2022-08-16 02:05:05+00:00   
18   2670726740  1559930024303759360 2022-08-17 15:47:54+00:00   

                                                  text                   id  \
252  - O Brasil já tem sua carta pela democracia: a...  1557892373379702784   
167  Relação amistosa com os Estados Unidos, tudo b...  1559374204163743744   
6    A cartilha "Lula Play", sugere o fortaleciment...  1560226397930536960   
176  Vou acabar com a molecagem política de humilha...  1559360566141001728   
18   Hoje vendemos metade dos carros que vendíamos ...  1559930024303759360   

     candidato  
252  bolsonaro  
167       ciro  
6         lula  
176       ciro  
18        lula  


Downloading: 100%|██████████| 190/190 [00:00<00:00, 215kB/s]
Downloading: 100%|██████████| 3.79k/3.79k [00:00<00:00, 2.84MB/s]
Downloading: 100%|██████████| 122/122 [00:00<00:00, 146kB/s]
Downloading: 100%|██████████| 471M/471M [00:23<00:00, 20.1MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 51.8kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:00<00:00, 5.14MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 270kB/s]
Downloading: 100%|██████████| 9.08M/9.08M [00:01<00:00, 7.44MB/s]
Downloading: 100%|██████████| 480/480 [00:00<00:00, 497kB/s]
Downloading: 100%|██████████| 14.8M/14.8M [00:01<00:00, 9.80MB/s]
Batches: 100%|██████████| 10/10 [00:18<00:00,  1.90s/it]
2022-08-24 19:17:11,319 - BERTopic - Transformed documents to Embeddings
2022-08-24 19:17:21,714 - BERTopic - Reduced dimensionality
2022-08-24 19:17:21,933 - BERTopic - Clustered reduced embeddings


    Topic  Count                                         Name
0      -1     68                              -1_que_de_do_em
1       0     55                           0_co_https_ciro_rt
2       1     22                            1_08_2022_ciro_co
3       2     16                          2_de_redução_não_em
4       3     13                           3_ano_ton_de_safra
5       4     12                   4_lula_equipelula_co_https
6       5     12                   5_programa_renda_de_mínima
7       6     11                 6_brasil_recuperar_para_mais
8       7     11              7_carta_democracia_que_saudades
9       8     11                  8_brasil_país_esperança_pro
10      9      8                            9_deus_se_você_os
11     10      8                       10_voto_eu_eleição_não
12     11      8                     11_hashtag_https_co_hoje
13     12      7                      12_gente_que_gerar_país
14     13      7             13_ele_bolsonaro_porque_mentiras
15     1

##Imports

In [5]:
tweets.head()


,author_id,conversation_id,created_at,text,id,candidato,topico
252,1.283729e+08,1.557892e+18,2022-08-12 00:51:00+00:00,- O Brasil já tem sua carta pela democracia: a...,1.557892e+18,bolsonaro,-1_que_de_do_em
167,3.337476e+07,1.559374e+18,2022-08-16 02:59:16+00:00,"Relação amistosa com os Estados Unidos, tudo b...",1.559374e+18,ciro,-1_que_de_do_em
6,2.670727e+09,1.560226e+18,2022-08-18 11:25:35+00:00,"A cartilha ""Lula Play"", sugere o fortaleciment...",1.560226e+18,lula,0_co_https_ciro_rt
176,3.337476e+07,1.559361e+18,2022-08-16 02:05:05+00:00,Vou acabar com a molecagem política de humilha...,1.559361e+18,ciro,16_governador_amigo_prefeito_alexandrekalil
18,2.670727e+09,1.559930e+18,2022-08-17 15:47:54+00:00,Hoje vendemos metade dos carros que vendíamos ...,1.559930e+18,lula,5_programa_renda_de_mínima


In [39]:
tweets2 = tweets[tweets['topico'] == '16_governador_amigo_prefeito_alexandrekalil']
tweets2.head()

,author_id,conversation_id,created_at,text,id,candidato,topico
176,3.337476e+07,1.559361e+18,2022-08-16 02:05:05+00:00,Vou acabar com a molecagem política de humilha...,1.559361e+18,ciro,16_governador_amigo_prefeito_alexandrekalil
12,2.670727e+09,1.560021e+18,2022-08-17 21:47:40+00:00,Artes pelo #BrasilDaEsperança!🇧🇷 Desenho da @a...,1.560021e+18,lula,16_governador_amigo_prefeito_alexandrekalil
147,3.337476e+07,1.559633e+18,2022-08-16 20:08:37+00:00,👍 GARANTIA – O programa será um direito consti...,1.559633e+18,ciro,16_governador_amigo_prefeito_alexandrekalil
138,3.337476e+07,1.559976e+18,2022-08-17 18:48:38+00:00,Quando o discurso da esquerda se assemelha ao ...,1.559976e+18,ciro,16_governador_amigo_prefeito_alexandrekalil
14,2.670727e+09,1.559998e+18,2022-08-17 20:18:08+00:00,"Eu sou candidato do povo brasileiro, e quero t...",1.559998e+18,lula,16_governador_amigo_prefeito_alexandrekalil


In [6]:
import re
import pandas as pd
import json
import nltk
import numpy as np

#Imports similaridade
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extração de Dados

In [ ]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [ ]:
def normaliza(df, coluna_alvo='text'):
  coluna_normal = coluna_alvo+"_normal"
  df[coluna_normal] = df[coluna_alvo].str.lower()
  df[coluna_normal].replace(to_replace = '[^a-zA-ZÀ-ÖØ-öø-ÿ\s]', value='', regex = True, inplace = True)
  return df
  
def tokeniza(df, coluna_alvo='text_normal'):
  stop_words = nltk.corpus.stopwords.words('portuguese')
  todos_tokens = []
  todos_textos_tokens = []

  for texto_normal in df[coluna_alvo].to_list():
    tokens = nltk.word_tokenize(texto_normal, language="portuguese")
    tokens_filtrados = [token for token in tokens if token not in stop_words]
    todos_tokens.append(tokens_filtrados)
    texto_tokens = ' '.join(tokens_filtrados)
    todos_textos_tokens.append(texto_tokens)


  df_tokens = pd.DataFrame({'tokens': todos_tokens, 'texto_tokens': todos_textos_tokens})
  df_concat = pd.concat([df, df_tokens], axis="columns")

  return df_concat


def normaliza_tokeniza(df, coluna_alvo = 'text'):
  df_tokens = tokeniza(normaliza(df, coluna_alvo), coluna_alvo)
  return df_tokens

  #word_regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+" #para capturarmos palavras dentro do tweet
  #df['text'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'], flags=re.MULTILINE)  #para remover os links do campo de texto do tweet

# Modelagem de Tópicos

*texto em itálico*# Análise de Sentimentos

# Similaridade de Textos

In [179]:
def monta_docs_tweets(df, topico, coluna_alvo, candidatos):
    lista_tweets = []
    #df_topico = df[df['topico'] == topico]
    df_topico = df

    for cand in candidatos:
        df_cand = df_topico[df_topico['candidato'] == cand]
        lista_tweets_cand = df_cand[coluna_alvo].to_list()
        lista_tweets.append(lista_tweets_cand)
    
    docs = np.concatenate(lista_tweets)
    return docs

def monta_docs_concat_topico(df, topico, coluna_alvo, candidatos):
    docs = []
    df_topico = df[(df['topico'] == topico)]
    for cand in candidatos:
        df_topico_cand = df_topico[df_topico['candidato'] == cand]
        texto = ' '.join(df_topico_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs


def monta_docs_concat_candidato(df, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        texto = ' '.join(df_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')

def normaliza_tweet(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet, re.I|re.A)
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = nltk.word_tokenize(tweet)
    tokens_filtrados = [token for token in tokens if token not in stop_words_ptbr]
    tweet_tokens = ' '.join(tokens_filtrados)
    return tweet_tokens

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def calcula_similaridade_topico(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_topico(df, topico, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)

    return df_sim

def calcula_similaridade_candidatos(df, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_candidato(df, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)

    return df_sim

def calcula_similaridade_topicos(df, topicos, coluna_alvo, candidatos):
    dfs_sim = []
    for topico in topicos:
        dfs_sim.append(calcula_similaridade_topico(df, topico, coluna_alvo, candidatos))
        
    sim_dict = dict(zip(topicos, dfs_sim))
    return sim_dict




In [166]:
df_freq = model.get_topic_info()
topicos_mais_frequentes = df_freq['Name'].to_list()[5:7]
topicos_mais_frequentes

['4_lula_equipelula_co_https', '5_programa_renda_de_mínima']

In [177]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "text", candidatos)
medias = []
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    media = np.mean(analise_similaridade[topico][0])
    print("analise_similaridade[topico][0]")
    print(analise_similaridade[topico][0])
    medias.append(media)
    print('----------------------------------')


print(medias)

Para tópico = 4_lula_equipelula_co_https
          0         1         2
0  1.000000  0.000000  0.005438
1  0.000000  1.000000  0.012693
2  0.005438  0.012693  1.000000
analise_similaridade[topico][0]
0    1.000000
1    0.000000
2    0.005438
Name: 0, dtype: float64
----------------------------------
Para tópico = 5_programa_renda_de_mínima
          0         1         2
0  1.000000  0.026211  0.018030
1  0.026211  1.000000  0.014704
2  0.018030  0.014704  1.000000
analise_similaridade[topico][0]
0    1.000000
1    0.026211
2    0.018030
Name: 0, dtype: float64
----------------------------------
[0.3351460717838304, 0.34808040573777205]


In [162]:
tweets[tweets['topico'] == "5_programa_renda_de_mínima"]

,author_id,conversation_id,created_at,text,id,candidato,topico
18,2.670727e+09,1.559930e+18,2022-08-17 15:47:54+00:00,Hoje vendemos metade dos carros que vendíamos ...,1.559930e+18,lula,5_programa_renda_de_mínima
77,2.670727e+09,1.558622e+18,2022-08-14 01:09:45+00:00,Um país com a riqueza do Brasil pode acabar co...,1.558622e+18,lula,5_programa_renda_de_mínima
242,1.283729e+08,1.558442e+18,2022-08-13 13:16:08+00:00,"- Em instantes, às 10h30, estarei ao vivo com ...",1.558442e+18,bolsonaro,5_programa_renda_de_mínima
222,1.283729e+08,1.559550e+18,2022-08-16 14:50:30+00:00,https://t.co/o0VsJIXXiG,1.559553e+18,bolsonaro,5_programa_renda_de_mínima
212,1.283729e+08,1.559858e+18,2022-08-17 11:00:34+00:00,"- Na contramão de outros governos, o estímulo ...",1.559858e+18,bolsonaro,5_programa_renda_de_mínima
264,1.283729e+08,1.557123e+18,2022-08-09 21:52:31+00:00,- Só os investimentos já garantidos nos último...,1.557123e+18,bolsonaro,5_programa_renda_de_mínima
60,2.670727e+09,1.559313e+18,2022-08-15 22:56:23+00:00,"Nós nordestinos, quando a gente se perde, a ge...",1.559313e+18,lula,5_programa_renda_de_mínima
71,2.670727e+09,1.558840e+18,2022-08-14 19:11:03+00:00,@tiamaoficial Um abraço pra você e sua família.,1.558894e+18,lula,5_programa_renda_de_mínima
192,3.337476e+07,1.558876e+18,2022-08-14 18:00:00+00:00,"Conversando com @caiofabiovvtv, perguntei qual...",1.558876e+18,ciro,5_programa_renda_de_mínima
182,3.337476e+07,1.559350e+18,2022-08-16 01:24:00+00:00,O que está acontecendo no Brasil é que está se...,1.559350e+18,ciro,5_programa_renda_de_mínima


In [181]:
analise_similaridade2 = calcula_similaridade_candidatos(tweets, "text", candidatos)
analise_similaridade2

,0,1,2
0,1.000000,0.320783,0.381142
1,0.320783,1.000000,0.291850
2,0.381142,0.291850,1.000000
